In [31]:
import warnings

import pandas as pd

from jre_utils.datapath import factor_data_paths, get_derived_csv_path
from jre_utils.config import asset_types
from jre_utils.visualize import plot_time_series
from jre_utils.process import (
    get_most_active_municipalities,
    get_highest_growth_municipalities,
    get_cumulative_growth,
)

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [79]:
factor = "taxable_income"
factor_column = "taxable_income_growth"

In [80]:
asset_type = "building"
# granularity_columns = ["area"]
granularity_columns = ["area", "area_code"]
group_by_columns = granularity_columns + ["year"]
display_columns = ["unit_price", "total_traded_area", "count"]

label = asset_types[asset_type]["label"]

# metric = "unit_price_wmean"
metric = "unit_price_wmedian"
# metric = "unit_price_mean"
# metric = "unit_price_median"
metric_pct_chg = metric + "_pct_chg"
upcoming_metric = "upcoming_" + metric
upcoming_metric_pct_chg = "upcoming_" + metric_pct_chg

In [86]:
core_df_path = get_derived_csv_path(asset_type)
factor_path = factor_data_paths["processed"][factor]["municipality"]

df = pd.read_csv(core_df_path)
factor_df = pd.read_csv(factor_path)

# time box
start_year = 2010
end_year = 2022
df = df[(df["year"] >= start_year) & (df["year"] <= end_year)]

df = df.merge(factor_df, on=group_by_columns, how="left")
df = df.sort_values(by=["year", "area_code"])

# prepare metrics
df[metric_pct_chg] = df.groupby(granularity_columns)[metric].pct_change(fill_method="ffill").fillna(0) # fill na with 0 for visualizations

# set up target variables
df[upcoming_metric] = df.groupby(granularity_columns)[metric].shift(-1)
df[upcoming_metric_pct_chg] = df.groupby(granularity_columns)[metric_pct_chg].shift(-1)

In [87]:
df[df["area_code"] == 13101] # Tokyo-to Chiyoda-ku

,year,area_code,area,unit_price_wmean,unit_price_wmedian,unit_price_mean,unit_price_median,total_traded_area,count,taxable_income,taxpayer_count,taxable_income_per_taxpayer,taxable_income_growth,taxable_income_per_taxpayer_growth,unit_price_wmedian_pct_chg,upcoming_unit_price_wmedian,upcoming_unit_price_wmedian_pct_chg
2434,2010,13101,Tokyo-to Chiyoda-ku,4.694486e+06,3.333333e+06,2.883100e+06,2.083035e+06,10700.0,46.0,217272181.0,28171.0,7712.618686,-0.034085,-0.044303,0.000000,2.000000e+06,-0.400000
2379,2011,13101,Tokyo-to Chiyoda-ku,2.348582e+06,2.000000e+06,1.944476e+06,1.545455e+06,13575.0,61.0,233634297.0,28440.0,8214.989346,0.075307,0.065136,-0.400000,1.911478e+06,-0.044261
2322,2012,13101,Tokyo-to Chiyoda-ku,2.684736e+06,1.911478e+06,1.964714e+06,1.488889e+06,10810.0,57.0,219420257.0,28761.0,7629.089983,-0.060839,-0.071321,-0.044261,2.916571e+06,0.525820
2263,2013,13101,Tokyo-to Chiyoda-ku,3.266448e+06,2.916571e+06,2.702967e+06,2.162162e+06,12190.0,63.0,233174299.0,29730.0,7843.064211,0.062684,0.028047,0.525820,3.460407e+06,0.186464
2205,2014,13101,Tokyo-to Chiyoda-ku,3.575147e+06,3.460407e+06,2.645518e+06,2.037500e+06,15330.0,68.0,280560533.0,31214.0,8988.291568,0.203222,0.146018,0.186464,3.000000e+06,-0.133050
2148,2015,13101,Tokyo-to Chiyoda-ku,3.761074e+06,3.000000e+06,2.865819e+06,2.124060e+06,11355.0,70.0,279917309.0,32993.0,8484.142364,-0.002293,-0.056090,-0.133050,3.562819e+06,0.187606
2090,2016,13101,Tokyo-to Chiyoda-ku,3.597246e+06,3.562819e+06,3.001553e+06,2.785714e+06,10530.0,64.0,314359478.0,34324.0,9158.591015,0.123044,0.079495,0.187606,4.118628e+06,0.156003
2032,2017,13101,Tokyo-to Chiyoda-ku,4.685343e+06,4.118628e+06,4.216382e+06,3.699248e+06,11530.0,66.0,333664476.0,35326.0,9445.294571,0.061411,0.031304,0.156003,3.158306e+06,-0.233166
1975,2018,13101,Tokyo-to Chiyoda-ku,4.059672e+06,3.158306e+06,3.767976e+06,3.061189e+06,11580.0,66.0,362690825.0,36299.0,9991.758037,0.086993,0.057856,-0.233166,4.417898e+06,0.398819
1917,2019,13101,Tokyo-to Chiyoda-ku,5.296680e+06,4.417898e+06,3.891026e+06,3.095455e+06,10995.0,64.0,412894018.0,38175.0,10815.822344,0.138419,0.082474,0.398819,4.112103e+06,-0.069217


In [88]:
comparables = [
    13102, # Tokyo-to Chuo-ku
    13103, # Tokyo-to Minato-ku
    15461, # Niigata-ken Yuzawa-Machi
    20321 # Nagano-ken Karuisawa-machi
]

frequency_n = 500
return_n = 10
factor_n = 10

frequency_df = get_most_active_municipalities(
    df.copy(), n=frequency_n, keep=comparables
)

return_df = get_highest_growth_municipalities(
    frequency_df.copy(),
    column=metric_pct_chg,
    cumulative_column="return_growth",
    end_year=2021,
    n=return_n,
    keep=comparables,
)

return_df["factor_growth"] = get_cumulative_growth(return_df.copy(), factor_column)

factor_growth_df = get_highest_growth_municipalities(
    frequency_df.copy(),
    column=factor_column,
    cumulative_column="factor_growth",
    end_year=2021,
    n=factor_n,
    keep=comparables,
)

factor_growth_df["return_growth"] = get_cumulative_growth(
    factor_growth_df.copy(), metric_pct_chg
)

# Tokyo Area Codes are 13000 + n. Therefore, if area_code // 1000 == 13, it is in Tokyo
tokyo_df = frequency_df[frequency_df["area_code"] // 1000 == 13]

In [90]:
plot_time_series(
    factor_growth_df.copy(),
    "return_growth",
    group_by_columns,
    granularity_columns,
    f"Cumulative Price Growth for top and bottom {return_n} municipalities",
    visible="legendonly",
)

In [85]:
plot_time_series(
    factor_growth_df.copy(),
    "factor_growth",
    group_by_columns,
    granularity_columns,
    f"cumulative {factor_column} for top and bottom {return_n} municipalities",
    # visible="legendonly",
)

In [58]:
plot_time_series(
    factor_growth_df.copy(),
    "count",
    group_by_columns,
    granularity_columns,
    f"Count for top and bottom {return_n} municipalities",
    # visible="legendonly",
)

In [12]:
# See if you can make a liquid long short equity strategy using REITs

# MVP
# Validating